In [7]:
import cv2
import mediapipe as mp
from mediapipe.tasks.python import vision
from mediapipe.tasks.python import BaseOptions
import time

result_list = []

# Función callback para procesar los resultados
def res_callback(result, output_image, timestamp_ms):
     result_list.append(result)

# Especificar la configuración
options = vision.PoseLandmarkerOptions(
     base_options=BaseOptions(model_asset_path="C:/Users/maria/Desktop/models/pose_landmarker_lite.task"),
     running_mode=vision.RunningMode.LIVE_STREAM,
     result_callback=res_callback)
landmarker = vision.PoseLandmarker.create_from_options(options)

# Leer el video de entrada
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

while True:
    ret, frame = cap.read()
    if ret == False:
        break
    
    h, w, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_rgb = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

    # Obtener los resultados
    landmarker.detect_async(frame_rgb, time.time_ns() // 1_000_000)

    if result_list:
        for lm in result_list[0].pose_landmarks:
            for each_lm in lm:
                if each_lm.visibility > 0.9:
                    x_each_lm = int(each_lm.x * w)
                    y_each_lm = int(each_lm.y * h)
                    cv2.circle(frame, (x_each_lm, y_each_lm), 3, (0, 255, 255), -1)
                        
        result_list.clear()
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import mediapipe as mp
from mediapipe.tasks.python import vision
from mediapipe.tasks.python import BaseOptions
import numpy as np

def calculate_distance(p1, p2):
    """Calcula la distancia euclidiana entre dos puntos (x, y)."""
    p1 = np.array(p1)
    p2 = np.array(p2)
    return np.linalg.norm(p1 - p2)

# ----- CONFIGURACIÓN DEL MODELO -----
options = vision.PoseLandmarkerOptions(
     base_options=BaseOptions(model_asset_path="C:/Users/maria/Desktop/models/pose_landmarker_lite.task"),
     running_mode=vision.RunningMode.VIDEO)
landmarker = vision.PoseLandmarker.create_from_options(options)

# ----- LECTURA DE IMAGEN PNG (SIN RESIZE) -----
image = cv2.imread("C:/Users/maria/Desktop/models/gyomei.png", cv2.IMREAD_UNCHANGED)
i = 0  # contador para deslizar la imagen

# ----- ENTRADA DE CÁMARA -----
cap = cv2.VideoCapture(0)   # <--- AQUÍ USAMOS LA WEBCAM
fps = 30

frame_index = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    output_frame = frame.copy()
    h, w, _ = frame.shape

    # Conversión a RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_rgb = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

    # Marca temporal (tú usas vídeo → simulo fps fijo)
    frame_timestamp_ms = int(1000 * frame_index / fps)
    frame_index += 1

    # Detección
    pose_landmarker_result = landmarker.detect_for_video(frame_rgb, frame_timestamp_ms)

    # Colores
    wrist_text_color = (255, 255, 255)
    index_text_color = (255, 255, 255)
    chest_text_color = (255, 255, 255)

    if pose_landmarker_result.pose_landmarks:
        for lm in pose_landmarker_result.pose_landmarks:
            # Oreja izquierda y derecha
            left_ear = int(lm[7].x * w), int(lm[7].y * h)
            right_ear = int(lm[8].x * w), int(lm[8].y * h)

            # Muñecas
            left_wrist = int(lm[15].x * w), int(lm[15].y * h)
            right_wrist = int(lm[16].x * w), int(lm[16].y * h)

            # Índices
            left_index = int(lm[19].x * w), int(lm[19].y * h)
            right_index = int(lm[20].x * w), int(lm[20].y * h)

            # Hombros
            left_shoulder = int(lm[11].x * w), int(lm[11].y * h)
            right_shoulder = int(lm[12].x * w), int(lm[12].y * h)

            # Caderas
            left_hip = int(lm[23].x * w), int(lm[23].y * h)
            right_hip = int(lm[24].x * w), int(lm[24].y * h)

            # Distancias
            dist_wrist = calculate_distance(left_wrist, right_wrist)
            dist_ear = calculate_distance(left_ear, right_ear)
            dist_index = calculate_distance(left_index, right_index)

            wrist_text_color = (253, 81, 139) if dist_wrist < dist_ear else (255, 255, 255)
            index_text_color = (253, 81, 139) if dist_index < dist_ear else (255, 255, 255)

            # Dibujar
            cv2.circle(output_frame, left_wrist, 6, (0, 255, 255), -1)
            cv2.circle(output_frame, right_wrist, 6, (128, 255, 0), -1)
            cv2.line(output_frame, left_wrist, right_wrist, (253, 81, 139), 2)

            cv2.circle(output_frame, left_index, 6, (255, 255, 0), -1)
            cv2.circle(output_frame, right_index, 6, (255, 0, 255), -1)
            cv2.line(output_frame, left_index, right_index, (253, 81, 139), 2)

            # ------- ÁREA DEL PECHO -------
            mid_left_chest = (
                left_shoulder[0] + (left_hip[0] - left_shoulder[0]) // 2,
                left_shoulder[1] + (left_hip[1] - left_shoulder[1]) // 2
            )
            mid_right_chest = (
                right_shoulder[0] + (right_hip[0] - right_shoulder[0]) // 2,
                right_shoulder[1] + (right_hip[1] - right_shoulder[1]) // 2
            )

            mask = np.zeros_like(frame)
            chest_polygon = np.array([[right_shoulder], [left_shoulder], [mid_left_chest], [mid_right_chest]])
            cv2.fillPoly(mask, pts=[chest_polygon], color=(253, 81, 139))

            inside_left = cv2.pointPolygonTest(chest_polygon, left_wrist, False)
            inside_right = cv2.pointPolygonTest(chest_polygon, right_wrist, False)
            chest_text_color = (253, 81, 139) if inside_left > 0 and inside_right > 0 else (255, 255, 255)

            output_frame = cv2.addWeighted(output_frame, 1, mask, 0.5, 0)

    # Texto
    cv2.putText(output_frame, "WRIST", (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, wrist_text_color, 2)
    cv2.putText(output_frame, "INDEX", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, index_text_color, 2)
    cv2.putText(output_frame, "CHEST AREA", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, chest_text_color, 2)

    # ------ Mostrar GYOMEI ------
    if wrist_text_color == (253, 81, 139) and index_text_color == (253, 81, 139) and chest_text_color == (253, 81, 139):

        if i < image.shape[0]:
            i += 3

        M = np.float32([[1, 0, w // 2], [0, 1, h - i]])
        translated = cv2.warpAffine(image, M, (w, h))

        mask_gyomei = cv2.cvtColor(translated[:, :, 3], cv2.COLOR_GRAY2BGR)
        output_frame = cv2.subtract(output_frame, mask_gyomei)
        output_frame = cv2.add(output_frame, translated[:, :, :3])
    else:
        i = 0

    # Mostrar cámaras
    cv2.imshow("Camara", frame)
    cv2.imshow("Salida", output_frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [5]:
import cv2
import mediapipe as mp

# Inicializar MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
mp_drawing = mp.solutions.drawing_utils

# Usar la cámara (0 = cámara principal)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # MediaPipe requiere RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Procesar pose
    results = pose.process(frame_rgb)

    # Dibujar landmarks si existen
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=4, circle_radius=2),
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=6, circle_radius=2)
        )

    # Mostrar
    cv2.imshow("Webcam - Pose", frame)

    # Salir con 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
